In [6]:
import matplotlib.pyplot as plt
import tensorflow as tf
import xgboost as xgb
import pandas as pd

In [5]:
lstm_reg= tf.keras.models.load_model('models/lstm_model_reg.h5')
bilstm_reg= tf.keras.models.load_model('models/bilstm_model_reg.h5')
gru_reg= tf.keras.models.load_model('models/gru_model_reg.h5')
cnn_lstm_reg= tf.keras.models.load_model('models/cnn_lstm_model_reg.h5')
ensemble_reg= tf.keras.models.load_model('models/ensemble_model.h5')
xgb_reg= xgb.Booster(model_file='models/xgb_model.json')

In [7]:
def predict_for_date(date, temperature, solar_generation, hour_of_day, is_holiday, dt, lstm_reg, le, scaler_cls, scaler_reg):
    new_data_class = pd.DataFrame({
        'temperature': [temperature],
        'solar_generation': [solar_generation],
        'hour_of_day': [hour_of_day],
        'load': [0],
        'is_holiday': [is_holiday]
    })

    new_data_reg = pd.DataFrame({
        'temperature': [temperature],
        'solar_generation': [solar_generation],
        'hour_of_day': [hour_of_day],
        'is_holiday': [is_holiday]
    })
    
    # Scale the data
    new_data_class_scaled = scaler_cls.transform(new_data_class)
    new_data_reg_scaled = scaler_reg.transform(new_data_reg)
    
    # Reshape for LSTM
    new_data_reg_lstm = new_data_reg_scaled.reshape((new_data_reg_scaled.shape[0], 1, new_data_reg_scaled.shape[1]))
    
    # Decision Tree predictions
    dt_class_pred = dt.predict(new_data_class_scaled)
    
    # LSTM predictions
    lstm_reg_pred = lstm_reg.predict(new_data_reg_lstm)
    
    predicted_compensation_method = le.inverse_transform(dt_class_pred)
    predicted_peak_supply = lstm_reg_pred

    return {
        'date': date,
        'hour_of_day': hour_of_day,
        'compensation_method': predicted_compensation_method[0],
        'peak_supply_needed': predicted_peak_supply[0][0]
    }

def generate_24_hour_predictions(timestamp, temperature, solar_generation, is_holiday, dt, lstm_reg, le, scaler_cls, scaler_reg):
    results = []
    hour = timestamp.hour
    prediction = predict_for_date(timestamp, temperature, solar_generation, hour, is_holiday, dt, lstm_reg, le, scaler_cls, scaler_reg)
    results.append(prediction)
    df_predictions = pd.DataFrame(results)
    return df_predictions

def generate_year_predictions(test_dataset, dt, lstm_reg, le, scaler_cls, scaler_reg):
    all_predictions = []
    for _, row in test_dataset.iterrows():
        timestamp = pd.to_datetime(row['timestamp'])
        temperature = row['temperature']
        solar_generation = row['solar_generation']
        is_holiday = row['is_holiday']
        daily_predictions = generate_24_hour_predictions(timestamp, temperature, solar_generation, is_holiday, dt, lstm_reg, le, scaler_cls, scaler_reg)
        all_predictions.append(daily_predictions)
    df_all_predictions = pd.concat(all_predictions, ignore_index=True)
    return df_all_predictions


In [8]:
testdf=pd.read_csv('dataset/24-hours testset - Copy.csv')
testdf.drop('electricity_demand', axis=1, inplace=True)
testdf['timestamp']=pd.to_datetime(testdf['timestamp'])

testdf.head()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,solar_generation
0,2024-01-01 00:00:00,0,0,0,28.534249,0,0.786249
1,2024-01-01 01:00:00,0,1,0,18.140664,0,7.331477
2,2024-01-01 02:00:00,0,2,0,19.539847,0,9.877753
3,2024-01-01 03:00:00,0,3,0,17.519129,0,1.215103
4,2024-01-01 04:00:00,0,4,0,18.255590,0,2.476539


In [ ]:
df_year_predictions = generate_year_predictions(
    test_dataset=testdf,
    dt=dt,
    lstm_reg=lstm_reg,
    le=le,
    scaler_cls=scaler_cls,
    scaler_reg=scaler_reg
)

df_year_predictions.head()